## Test environment

In [1]:
# Import modules
from multiplexdesigner.designer.design import design_primers
from multiplexdesigner.designer.multiplexpanel import panel_factory

In [2]:
# Get the mutation list (junctions) around which to design primer pairs
fasta_file = "/Users/ctosimsen/Documents/data/genomes/hg38/hg38.fa"
config_file = "../config/designer_default_config.json"
design_input_file = "../data/junctions.csv"

The `panel_factory` creates a panel object with suitable design regions obtained from the reference genome, as well a logger, which is passed through to write information to the log file. Both get then passed to the design engine, which will design candidate primers based on the chosen algorithm `simsen` or `primer3`.

In [3]:
panel = design_primers(
    panel=panel_factory(
        name="test_panel",
        genome="hg38",
        design_input_file=design_input_file,
        fasta_file=fasta_file,
        config_file=config_file,
        padding=200,
    ),
    method="simsen",
)

2026-01-29 16:44:47.060 | INFO     | multiplexdesigner.designer.multiplexpanel:panel_factory:827 - Creating multiplex panel: test_panel
2026-01-29 16:44:47.061 | INFO     | multiplexdesigner.config:load_config:373 - Loading config from: ../config/designer_default_config.json
2026-01-29 16:44:47.070 | INFO     | multiplexdesigner.designer.multiplexpanel:import_junctions_csv:276 - Successfully imported 20 junctions from ../data/junctions.csv
2026-01-29 16:44:47.071 | INFO     | multiplexdesigner.designer.multiplexpanel:merge_close_junctions:312 - Merging junctions within 120 bp on same chromosome...
2026-01-29 16:44:47.078 | DEBUG    | multiplexdesigner.designer.multiplexpanel:_merge_junctions_df:395 - Merged 2 junctions: CLTCL1_p.R354H_CLTCL1_p.R354H
2026-01-29 16:44:47.081 | INFO     | multiplexdesigner.designer.multiplexpanel:merge_close_junctions:327 - After merging: 19 junctions remain
2026-01-29 16:44:47.082 | INFO     | multiplexdesigner.designer.multiplexpanel:extract_design_regi

In [ ]:
panel.save_candidate_primers_to_fasta("./test.fa")

In [ ]:
panel.unique_primer_map

In [ ]:
from multiplexdesigner.blast.specificity import run_specificity_check

run_specificity_check(panel, "./", fasta_file)

In [ ]:
panel.junctions[4].primer_pairs[0].off_target_products

## Exploring the panel object

Each panel contains one or more junctions/targets. Primers are designed left and right of each target first and then suitable primers pairs for each target are selcted from the designs.

Individual primer picking evaluates basic properties (length, Gc-content, Tm, fraction bound) as well thermodynamic properties (self and 3' complementarity). Primers receive a penalty based on the configuration file provided. suitable pairs are then selected by finding pairs of primers that can form a permissible amplicon. For ctDNA applications, where short amplicons are preferable, most pairs will be removed due to too long amplicons.

In [ ]:
panel.junctions[0].primer_pairs[2].forward

In [ ]:
panel.junctions[0].primer_pairs[2].reverse

In [ ]:
panel.junctions[0].primer_pairs[0]

In [ ]:
panel.junctions[0].primer_pairs[0].amplicon_sequence

In [ ]:
(
    len(panel.junctions[0].primer_pairs[1].amplicon_sequence)
    - len("AGACATTCTCACCTTGACATCTCAG")
    - len("TGCGCTGCCATGACTGTCA")
)

In [ ]:
for junction in panel.junctions:
    if hasattr(junction, "design_region"):
        print(f"\n{junction.name}:")
        print(f"  Region: {junction.design_region} ")

In [ ]:
import primer3

tm_result = primer3.bindings.calcHeterodimer(
    "AGACATTCTCACCTTGACATCTCAG", "TGCGCTGCCATGACTGTC", output_structure=True
)

In [ ]:
tm_result.tm

In [ ]:
tm_result.structure_found

In [ ]:
tm_result.dg

In [ ]:
tm_result.ascii_structure_lines

## Testing Thermodynamics

In [4]:
from multiplexdesigner.designer.thal import seqtm

In [5]:
panel.junctions[0]

Junction(WLS_p.I360N , chr1:68144579-68144579)

In [6]:
panel.junctions[0].design_region

'GGTTTTACGGAAAGAGAAAATTTAGTCCATTTGGCCAGGTGATATCAAGACCCAGAAAAATATGGCTTGACCAGCTGTCCAGAATTATGGCCTCTCTCCTCCTTTCCTCAGTGGCTCTATTGAATTTCTGTGCAGGGTAGAGGGATCTCTGCAGAGACATTCTCACCTTGACATCTCAGGGTCCACTTACCTGACTAACGATGAAGAAGATGACAGTCATGGCAGCGCAGGCCAAGGTGATAAGCATGAGGAACTTGAACCTAAAAATTAGCCCCTATTAGAAAAGAAAGAGTAGTTTAATACTCCATCAGCTACCAATCCTTTTCTCACTATGTAAATCTATAAAAAGCTTAATTTTAAAGAAATCTGTAAAACCAAATCCCTAAGAATGACAGTAAAAT'

In [12]:
panel.junctions[0].primer_pairs[500].forward

Primer(name='WLS_p.I360N _1286_forward', seq='ATTTCTGTGCAGGGTAGAGGGATCT', direction='forward', start=123, length=25, bound=99.5, tm=63.1, tm_primer3=None, gc=48.0, penalty=7.6, self_any_th=-38.67, self_end_th=-49.84, hairpin_th=0.0, end_stability=2.75, engine='custom')

In [ ]:
panel.junctions[0].primer_pairs[0].reverse

In [4]:
import math
from multiplexdesigner.designer.thal import calc_thermodynamics,divalent_to_monovalent, symmetry

In [37]:
GAS_CONSTANT = 1.987  # In cal/(K·mol)
T_KELVIN = 273.15

seq = "AAGTGGCAGCTGTGGCCCTGA"
dna_conc= 50.0
salt_conc = 50.0
divalent_conc = 1.5
dntp_conc = 0.8
dmso_conc = 0.0
dmso_fact = 0.6
formamide_conc = 0.0
annealing_temp = 60.0

# Validate and convert divalent to monovalent
dv_to_mv = divalent_to_monovalent(divalent_conc, dntp_conc)

# Count GC for formamide correction
gc_count = sum(1 for base in seq if base in "GC")
seq_len = len(seq)

# Calculate thermodynamics
dh, ds = calc_thermodynamics(seq)

delta_H = dh * -100.0  # Convert to cal/mol
delta_S = ds * -0.1  # Convert to cal/(K·mol)

K_mM = salt_conc + dv_to_mv

temp = annealing_temp + T_KELVIN

# The entropy correction is given by the following equation (SantaLucia 1998):
delta_S = delta_S + 0.368 * (seq_len - 1) * math.log(K_mM / 1000.0)

ddG = delta_H - temp * delta_S
ka = math.exp((delta_S / GAS_CONSTANT) - (delta_H / (GAS_CONSTANT * temp)))


if symmetry(seq):
    Tm = delta_H / (delta_S + GAS_CONSTANT * math.log(dna_conc / 1e9)) - T_KELVIN
    bound = (1 / (1 + math.sqrt(1 / ((dna_conc / 1e9) * ka)))) * 100
else:
    Tm = delta_H / (delta_S + GAS_CONSTANT * math.log(dna_conc / 4e9)) - T_KELVIN
    bound = (1 / (1 + math.sqrt(1 / ((dna_conc / 4e9) * ka)))) * 100

# Apply DMSO and formamide corrections
if dmso_conc > 0.0:
    Tm -= dmso_conc * dmso_fact
    Tm += (0.453 * gc_count / seq_len - 2.88) * formamide_conc


In [38]:
Tm

66.2754397691121

In [39]:
bound

90.73613668470384

In [40]:
delta_S

-445.243241537775

In [41]:
ddG

-15067.214081690268